In [1]:
import pandas as pd

# Load the dataset
coords_file_path = 'coords.csv'
coords_data = pd.read_csv(coords_file_path)

# Display the first few rows of the dataset
coords_data.head()


,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z18,x19,y19,z19,x20,y20,z20,x21,y21,z21
0,A,0.530803,0.581163,-1.472346e-06,0.413990,0.501149,-0.018073,0.355407,0.369223,-0.029657,...,-0.057639,0.605741,0.320499,-0.091212,0.588447,0.400885,-0.071050,0.594126,0.455363,-0.044118
1,A,0.503664,0.616673,-1.548741e-06,0.377512,0.536617,-0.016148,0.317852,0.408111,-0.028432,...,-0.057488,0.561715,0.359349,-0.085140,0.546541,0.440563,-0.067003,0.551058,0.498263,-0.044481
2,A,0.267808,0.670679,-8.506087e-07,0.187360,0.610215,-0.020350,0.141140,0.503103,-0.030225,...,-0.043488,0.369881,0.461587,-0.075160,0.362212,0.517269,-0.059621,0.349507,0.565708,-0.036153
3,A,0.289091,0.751779,-1.795283e-06,0.167962,0.654705,-0.030267,0.100007,0.499485,-0.048589,...,-0.059609,0.393687,0.455553,-0.107050,0.365366,0.547497,-0.087988,0.356335,0.612805,-0.058007
4,A,0.285160,0.762803,-1.769840e-06,0.161197,0.667723,-0.030836,0.094453,0.512100,-0.048447,...,-0.059550,0.388296,0.466932,-0.105277,0.361084,0.560643,-0.085365,0.352804,0.623975,-0.055310


In [2]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# Separating features and target variable
X = coords_data.drop('class', axis=1)
y = coords_data['class']

'''# Normalizing the coordinate data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)'''

# Encoding the class labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Splitting the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=42)


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


# Reshape input to be [samples, time steps, features] which is required for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((51233, 1, 63), (12809, 1, 63), (51233,), (12809,))

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(20))
model.add(Dense(10, activation='relu'))
model.add(Dense(len(np.unique(y_train)), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/10


2023-11-16 00:46:09.896028: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


801/801 [==============================] - 2s 1ms/step - loss: 1.8991 - accuracy: 0.4298 - val_loss: 1.0192 - val_accuracy: 0.6748
Epoch 2/10
801/801 [==============================] - 1s 1ms/step - loss: 0.6981 - accuracy: 0.8231 - val_loss: 0.5021 - val_accuracy: 0.8827
Epoch 3/10
801/801 [==============================] - 1s 1ms/step - loss: 0.4132 - accuracy: 0.8953 - val_loss: 0.3405 - val_accuracy: 0.9215
Epoch 4/10
801/801 [==============================] - 1s 1ms/step - loss: 0.2946 - accuracy: 0.9267 - val_loss: 0.2747 - val_accuracy: 0.9244
Epoch 5/10
801/801 [==============================] - 1s 1ms/step - loss: 0.2368 - accuracy: 0.9396 - val_loss: 0.2190 - val_accuracy: 0.9474
Epoch 6/10
801/801 [==============================] - 1s 1ms/step - loss: 0.2019 - accuracy: 0.9470 - val_loss: 0.1973 - val_accuracy: 0.9554
Epoch 7/10
801/801 [==============================] - 1s 1ms/step - loss: 0.1796 - accuracy: 0.9529 - val_loss: 0.1747 - val_accuracy: 0.9575
Epoch 8/10
801/80